### [T5](https://github.com/google-research/text-to-text-transfer-transformer)
- **Text-To-Text Transfer Transformer**
- A unified framework that converts every language problem into a text-to-text format.
- Achieves state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more.

### Multi Class vs Multi Label Classification
- **Multi Class** - There are multiple categories but each instance is assigned only one, therefore such problems are known as multi-class classification problem.

# Imports

The entire code is written using **PyTorch**.<br>
We'll be using the **transformers** library by [huggingface](https://github.com/huggingface/transformers) as they provide wrappers for multiple Transformer models.

In [1]:
!pip install transformers
!pip install pytorch-lightning --upgrade
!pip install sentencepiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.0.4
    Uninstalling pytorch-lightning-2.0.4:
      Successfully uninstalled pytorch-lightning-2.0.4


In [2]:
'''rom google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d sifalklioui/hatespeechdza
!mkdir data
!unzip hatespeechdza.zip -d ./data'''

'rom google.colab import files\n\nfiles.upload()\n! mkdir ~/.kaggle\n! cp kaggle.json ~/.kaggle/\n! chmod 600 ~/.kaggle/kaggle.json\n! kaggle datasets download -d sifalklioui/hatespeechdza\n!mkdir data\n!unzip hatespeechdza.zip -d ./data'

In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
import copy
from tqdm.notebook import tqdm
import gc
import random
import torch
import logging
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import pytorch_lightning as pl
from torch.optim import AdamW
from sklearn import metrics

from transformers import (
    T5Tokenizer,
    T5Model,
    T5ForConditionalGeneration,
    get_linear_schedule_with_warmup
)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
@dataclass
class Config:
    seed = 203
    data_folder = "../input/hatespeechdza"
    output_dir = './logs'
    model_name_or_path = 'UBC-NLP/AraT5v2-base-1024'
    src_max_length = 45
    tgt_max_length = 2
    add_special_tokens = True
    truncation = True
    return_tensors = 'pt'
    padding = "max_length"
    weight_decay=0.0
    adam_epsilon=1e-8
    warmup_steps=0
    train_batch_size=64
    eval_batch_size=64
    num_train_epochs=10
    gradient_accumulation_steps=16
    n_gpu=1
    fp_16= False, # if you want to enable 16-bit training then install apex and set this to true
    max_grad_norm=1.0 # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    learning_rate= float(3e-4)

config = Config()

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(config.seed)

# Dataset & Dataloader

Now, we'll create a custom Dataset class inherited from the PyTorch Dataset class. We'll be using the **T5 tokenizer** that returns **input_ids** and **attention_mask**.<br><br>
The custom Dataset class will return a dict containing - <br>

- src_input_ids
- src_attention_mask
- tgt_input_ids'
-tgt_attention_mask

In [6]:
import pandas as pd
from datasets import Dataset,load_dataset

class HateDetect():
    def __init__(self,config,tokenizer, part):

        self.config = config
        self.part = part
        self.tokenizer = tokenizer


        data_paths = {
            'train': config.data_folder + "/dataset_prep_train.csv",
            'test': config.data_folder + "/dataset_prep_test.csv",
            'val': config.data_folder + "/dataset_prep_val.csv"
        }
        path = data_paths.get(self.part,None)
        if path is not None:
            df = pd.read_csv(path)
            df['label'].replace({0:"normal",1:"hate"}, inplace = True)
            self.data = Dataset.from_pandas(df ,split=self.part)
        else:
            raise ValueError("Invalid value for self.part")


        self.dataset_scr,self.dataset_tgt = self.tokenize()

        # create funtion to tokenize data
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):

        source_ids = self.dataset_scr["input_ids"][idx].squeeze()
        target_ids = self.dataset_tgt["input_ids"][idx].squeeze()

        src_mask    = self.dataset_scr["attention_mask"][idx].squeeze()
        target_mask = self.dataset_tgt["attention_mask"][idx].squeeze()

        return {"source_ids": source_ids,
                "source_mask": src_mask,
                "target_ids": target_ids,
                "target_mask": target_mask}


    def tokenize(self):

        tokenizer_params = {
            "src": {
                "max_length": self.config.src_max_length,
                "add_special_tokens": self.config.add_special_tokens,
                "truncation": self.config.truncation,
                "return_tensors": self.config.return_tensors,
                "padding": self.config.padding
            },
            "tgt": {
                "max_length": self.config.tgt_max_length,
                "add_special_tokens": self.config.add_special_tokens,
                "truncation": self.config.truncation,
                "return_tensors": self.config.return_tensors,
                "padding": self.config.padding
            }
        }
        dataset_scr = self.tokenizer.batch_encode_plus(self.data['text'], **tokenizer_params["src"])
        dataset_tgt = self.tokenizer.batch_encode_plus(self.data['label'], **tokenizer_params["tgt"])
        return dataset_scr,dataset_tgt

def get_dataset(config,tokenizer,part):
    return HateDetect(config,tokenizer,part)

In [7]:
logger = logging.getLogger(__name__)

class DeviceCallback(pl.Callback):
    def on_batch_start(self, trainer, pl_module):
        assert next(pl_module.parameters()).device.type == "cuda"

class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.config.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [8]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath = config.output_dir, monitor="validation_epoch_average", mode="max", save_top_k=1
)
train_params = dict(
    devices=config.n_gpu,
    strategy="auto",
    accumulate_grad_batches=config.gradient_accumulation_steps,
    accelerator="gpu",
    max_epochs=config.num_train_epochs,
    precision= '16-mixed' if config.fp_16 else 32,
    gradient_clip_val=config.max_grad_norm,
    callbacks=[LoggingCallback(),checkpoint_callback,DeviceCallback()],
)

# Model

Coming to the most interesting part - the model architecture! We'll create a class named **Model**, inherited from **torch.nn.Module**.<br><br>

### Flow
- We initialize our pretrained T5 model with a Conditional Generation Head.
- Pass in the src & tgt, input_ids & attention_mask.
- The model returns the decoder generated output ids (predicted labels in textual format), which we need to decode further using the tokenizer.

In [9]:
!pip install wandb
!pip install lightning

from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="HTarabT5")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3                                                                                              │
│   4 from lightning.pytorch.loggers import WandbLogger                                            │
│   5                                                                                              │
│ ❱ 6 wandb_logger = WandbLogger(project="HTarabT5")                                               │
│   7                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:359 in __init__       │
│                                                                                                  │
│   356 │   │   # start wandb run (to create an attach_id for distributed modes)                   │
│   357 │   │   if _WANDB_GREATER_EQUAL_0_12_10:                                                   │
│   358 │   │   │   wandb.require("service")                                                       │
│ ❱ 359 │   │   │   _ = self.experiment                                                            │
│   360 │   │                                                                                      │
│   361 │   │   self._checkpoint_name = checkpoint_name                                            │
│   362                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/lightning/fabric/loggers/logger.py:118 in experiment     │
│                                                                                                  │
│   115 │   │   """                                                                                │
│   116 │   │   if rank_zero_only.rank > 0:                                                        │
│   117 │   │   │   return _DummyExperiment()                                                      │
│ ❱ 118 │   │   return fn(self)                                                                    │
│   119 │                                                                                          │
│   120 │   return experiment                                                                      │
│   121                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:407 in experiment     │
│                                                                                                  │
│   404 │   │   │   │   self._experiment = wandb._attach(attach_id)                                │
│   405 │   │   │   else:                                                                          │
│   406 │   │   │   │   # create new wandb process                                                 │
│ ❱ 407 │   │   │   │   self._experiment = wandb.init(**self._wandb_init)                          │
│   408 │   │   │   │                                                                              │
│   409 │   │   │   │   # define default x-axis                                                    │
│   410 │   │   │   │   if isinstance(self._experiment, (Run, RunDisabled)) and getattr(           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_ini

In [ ]:
!pip install torchmetrics

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = T5ForConditionalGeneration.from_pretrained(config.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(config.model_name_or_path)
        #self.f1 = torchmetrics.F1()
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

    def is_logger(self):
        return self.trainer.global_rank  <= 0

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
        ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
            )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100
        outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
        )
        #print(outputs.shape)

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("train/loss", loss)
        self.training_step_outputs.append(loss)
        return loss

    def on_train_epoch_end(self):
        epoch_average = torch.stack(self.training_step_outputs).mean()
        self.log("training_epoch_average", epoch_average, sync_dist=True, prog_bar=True, logger=True, on_epoch=True)
        self.training_step_outputs.clear()  # free memory

    def test_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.test_step_outputs.append(loss)
        return loss

    def on_test_epoch_end(self):
        epoch_average = torch.stack(self.test_step_outputs).mean()
        self.log("test_epoch_average", epoch_average, sync_dist=True, prog_bar=True, logger=True, on_epoch=True)
        self.test_step_outputs.clear()  # free memory

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        #y = batch['target_ids'].clone().detach()
        #lm_labels = y.clone().detach()
        #lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100
        self.validation_step_outputs.append(loss)
        
        return loss

    def on_validation_epoch_end(self):
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average, sync_dist=True, prog_bar=True, logger=True, on_epoch=True)
        self.validation_step_outputs.clear()  # free memory

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]

        optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.config.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.config.learning_rate, eps=self.config.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=self.config.warmup_steps, num_training_steps=self.trainer.estimated_stepping_batches)
        self.lr_scheduler = scheduler
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer],[scheduler]

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}
        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(config=self.config, tokenizer=self.tokenizer, part="train")
        return DataLoader(train_dataset, batch_size=self.config.train_batch_size, drop_last=True, shuffle=True,num_workers=2,pin_memory=True)
    def val_dataloader(self):
        val_dataset = get_dataset(config=self.config,tokenizer=self.tokenizer, part="val")
        return DataLoader(val_dataset, batch_size=self.config.eval_batch_size, num_workers=2,pin_memory=True)
    def test_dataloader(self):
        val_dataset = get_dataset(config=self.config,tokenizer=self.tokenizer, part="test")
        return DataLoader(val_dataset, batch_size=self.config.eval_batch_size, num_workers=2,pin_memory=True)

In [ ]:
model = T5FineTuner(config)

In [ ]:
trainer = pl.Trainer(**train_params,overfit_batches=0.1)

In [ ]:
trainer.fit(model)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(config.model_name_or_path)

In [ ]:
#model = T5FineTuner.load_from_checkpoint("/content/logs/epoch=9-step=80.ckpt",config=config)

In [ ]:
model.cuda()
outputs = []
targets = []
for batch in tqdm(loader):
    outs = model.model.generate(input_ids=batch['source_ids'].cuda(),
                              attention_mask=batch['source_mask'].cuda(),
                              max_length=2)

    dec = [tokenizer.decode(ids[ids > 1 ]) for ids in outs]
    target = [tokenizer.decode((ids[ids > 1 ])) for ids in batch["target_ids"]]

    outputs.extend(dec)
    targets.extend(target)

In [ ]:
print(metrics.classification_report(targets, outputs))

In [ ]:
import textwrap

In [ ]:
for i in range(32):
    lines = textwrap.wrap("Review:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual sentiment: %s" % targets[i])
    print("Predicted sentiment: %s" % dec[i])
    print("=====================================================================\n")